In [3]:
# Torch and GPT2 model
import torch
from torch.distributions import Categorical
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# logging
import logging
logging.basicConfig(level=logging.INFO)

# Widgets
from IPython.display import display
from ipywidgets import widgets

In [4]:
# Ignore some irrelevant warnings
import warnings
warnings.filterwarnings('ignore')

In [5]:
# load the model and set in eval mode 
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.eval();
model.to('cuda');

# initialize tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

INFO:transformers.configuration_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-config.json from cache at /home/albert/.cache/torch/transformers/4be02c5697d91738003fb1685c9872f284166aa32e061576bbe6aaeb95649fcf.4c1d7fc2ac6ddabeaf0c8bec2ffc7dc112f668f5871a06efcff113d2797ec7d5
INFO:transformers.configuration_utils:Model config GPT2Config {
  "_num_labels": 2,
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bad_words_ids": null,
  "bos_token_id": 50256,
  "decoder_start_token_id": null,
  "do_sample": false,
  "early_stopping": false,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "finetuning_task": null,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "is_decoder": false,
  "is_encoder_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_epsilon": 1e-05,
  "length_penalty": 1.0,
  "max_length": 20,
  "min_lengt

In [6]:
def predict_words(model, text, paragraph_len, temp = None):
    """
    Function that predicts next words given an input sentence
    
    Args:
      Model: The transformer model to be used
      Text : The input text
      paragraph_len: Amount of words to be added to original sequence
      Temp: Temperature used in sampling instead of argmax
    Returns:
      Prints the predicted sequence
    """
    
    # Encode the tokens and get em cudatorch-y
    indexed_tokens = tokenizer.encode(text)
    tokens_tensor = torch.tensor([indexed_tokens])
    tokens_tensor = tokens_tensor.to('cuda')
    
    # Predict stuff
    with torch.no_grad():
        for i in range(paragraph_len):
            
            # Pass through model
            outputs = model(tokens_tensor)
            predictions = outputs[0]
            
            if temp == None:
                # Get argmax
                predicted_index = torch.argmax(predictions[0,-1,:]).item()
            else:
                # Sample with temperature
                epsilon = 1e-8
                x = predictions[0,-1,:] * temp
                m = Categorical(logits = (x + epsilon))
                predicted_index = m.sample()
                #print(sum(m.probs))
                #print(sum(m.logits))
            
            # Add prediction to previous predictions
            predicted_index = torch.tensor([predicted_index]).to('cuda').unsqueeze(0)
            tokens_tensor = torch.cat((tokens_tensor, predicted_index), dim=1)
    
    # Squeeze and decode
    tokens_tensor = tokens_tensor.squeeze(0)
    predicted_text = tokenizer.decode(tokens_tensor.tolist())

    print(predicted_text)

In [7]:
# Initialize input widget and get standard text
inputText = widgets.Text(placeholder='Type something!')
text = "Who was Jim Henson ? Jim Henson was a"

# How many words we predict
paragraph_len = 40
temp = 3

# Text event handler
def getText(s):
    global text
    text = inputText.value
    
# Button eventhandler
def button_click(b):
    global text, model
    text = inputText.value
    predict_words(model, text, paragraph_len, temp)
    
# Textbox stuff
inputText.on_submit(getText)
display(inputText)
inputText
    
# Button stuff
button = widgets.Button(description="Submit!")
display(button)
button.on_click(button_click)

Text(value='', placeholder='Type something!')

Button(description='Submit!', style=ButtonStyle())

RuntimeError: CUDA error: unspecified launch failure

In [19]:
import numpy as np

def softmax(numbers):
    total = sum(np.exp(numbers))
    
    return [np.exp(num)/total for num in numbers]

In [21]:
a = np.array([1,2,3])
softmax(a)

[0.09003057317038046, 0.24472847105479767, 0.6652409557748219]

In [24]:
result = np.array([1,2,3])
for i in range(10):
    print(result)
    result = softmax(result)

[1 2 3]
[0.09003057317038046, 0.24472847105479767, 0.6652409557748219]
[0.2534976524899556, 0.29590914376846866, 0.45059320374157574]
[0.30663708811686846, 0.319921744103127, 0.3734411677800045]
[0.32441628763919456, 0.3287548004361928, 0.34682891192461257]
[0.33035860693641345, 0.33179498560077175, 0.3378464074628149]
[0.33234148068347963, 0.3328191919005808, 0.3348393274159394]
[0.3330026845016226, 0.33316180162232456, 0.3338355138760528]
[0.3332231135624029, 0.33327613928332245, 0.33350074715427475]
[0.3332965930208367, 0.33331426678153947, 0.33338914019762383]
